# Model Calibration / i.e. Parameter Estimation

Our models contain a lot of parameters that need to be set before the model can make useful predictions.
Some parameters in our models can be set directly from measurements, such as column lengths.
Other can not be measured directly, such as dispersion coefficients, and need to be estimated by comparing simulation predictions to experimental data.

For our target `LRM` with `SMA` model, these are the parameters we can measure externally:

```
inlet.flow_rate
inlet.c
lrm.length
lrm.diameter
sma.capacity
```

And these are the ones we need to fit:

```
lrm.axial_dispersion
lrm.total_porosity
sma.adsorption_rate
sma.desorption_rate
sma.characteristic_charge
sma.steric_factor
```

In [ ]:
from CADETProcess.processModel import Inlet, Outlet, ComponentSystem, LumpedRateModelWithoutPores, StericMassAction

component_system = ComponentSystem(["Salt", "ProteinA", "ProteinB", "ProteinC"])

inlet = Inlet(component_system, "inlet")
inlet.flow_rate = 1 / 60 / 1000 / 1000

outlet = Outlet(component_system, "outlet")

column = LumpedRateModelWithoutPores(component_system, "column")

column.total_porosity = 0.4
column.axial_dispersion = 1e-6
column.length = 0.014
column.diameter = 0.01
column.c = [50, 0, 0, 0]

binding = StericMassAction(component_system, "binding")
binding.is_kinetic = True
binding.adsorption_rate = [0, 1e-5, 1e-1, 1e-3]
binding.desorption_rate = [0, 1, 1, 1]
binding.characteristic_charge = [0, 5, 1, 7]
binding.steric_factor = [0, 1, 1, 1]
binding.capacity = 1200

column.binding_model = binding

column.q = [50, 0, 0, 0]

from CADETProcess.processModel import FlowSheet, Process

flow_sheet = FlowSheet(component_system, "flow_sheet")

flow_sheet.add_unit(inlet)
flow_sheet.add_unit(column)
flow_sheet.add_unit(outlet)

flow_sheet.add_connection(inlet, column)
flow_sheet.add_connection(column, outlet)

process = Process(flow_sheet, "process")

gradient_column_volumes = 15
gradient_duration = gradient_column_volumes * column.volume / inlet.flow_rate[0]

process.cycle_time = 90 + gradient_duration + 90

# add load
total_protein = 0.001  # mol
load_duration = 10  # sec
protein_concentration = 1.0

process.add_event(
    name="load",
    parameter_path="flow_sheet.inlet.c",
    state=[
        [50, 0, 0, 0],
        [protein_concentration, 0, 0, 0],
        [protein_concentration, 0, 0, 0],
        [protein_concentration, 0, 0, 0]
    ],
    time=0
)
# add wash
process.add_event(
    name="wash",
    parameter_path="flow_sheet.inlet.c",
    state=[
        [50, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ],
    time=10
)

start_concentration = 50
end_concentration = 1000
gradient_duration

slope = (end_concentration - start_concentration) / gradient_duration

# add gradient
process.add_event(
    name="gradient_start",
    parameter_path="flow_sheet.inlet.c",
    state=[
        [50, slope, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ],
    time=90
)

# stop gradient
process.add_event(
    name="gradient_end",
    parameter_path="flow_sheet.inlet.c",
    state=[
        [1000, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ],
    time=90 + gradient_duration
)

process.check_config()

We need to start somewhere.

What is the chromatographic experiment with the least parameters you can think of?

The experiment we ususally start with is a simple tracer pulse injection onto a chromatographic column with a non-binding molecule.

The only parameters that influence the elution are: `lrm.axial_dispersion` and `lrm.total_porosity`

Your colleagues from the lab have shared their lab measurement with you in `./experimental_data/tracer_1.xlsx`. We can use the `pandas` library to import and plot the data.

In [ ]:
import pandas as pd

data = pd.read_excel("experimental_data/15cv_gradient.xlsx", index_col=0)
data.plot()

## References

To quantify the difference between simulation and reference, **CADET-Process** provides a `comparison` module.

To properly work with **CADET-Process**, the experimental data needs to be converted to an internal standard.
The `reference` module provides different classes for different types of experiments.
For in- and outgoing streams of unit operations, the `ReferenceIO` class must be used.

In [ ]:
from CADETProcess.reference import ReferenceIO

reference = ReferenceIO("lab_data", data.index, data, component_system=component_system)

Similarly to the `SolutionIO` class, the `ReferenceIO` class also provides a plot method:

In [ ]:
reference.plot()

## Comparator

The `Comparator` class comparing the simulation output with experimental data. It provides several methods for visualizing and analyzing the differences between the data sets. Users can choose from a range of metrics to quantify the differences between the two data sets, such as sum squared errors or shape comparison.

In [ ]:
from CADETProcess.comparison import Comparator

comparator = Comparator("comparator_15cv")
comparator.add_reference(reference)

```{note}
It's also possible to add multiple references, e.g. for triplicate experiments or for different sensors.
```

## Difference Metrics
There are many metrics which can be used to quantify the difference between the simulation and the reference.
Most commonly, the sum squared error (SSE) is used.

However, SSE is often not an ideal measurement for chromatography.
Because of experimental non-idealities like pump delays and fluctuations in flow rate there is a tendency for the peaks to shift in time.
This causes the optimizer to favour peak position over peak shape and can lead for example to an overestimation of axial dispersion.

In contrast, the peak shape is dictated by the physics of the physico-chemical interactions while the position can shift slightly due to systematic errors like pump delays.
Hence, a metric which prioritizes the shape of the peaks being accurate over the peak eluting exactly at the correct time is preferable.
For this purpose, **CADET-Process** offers a `Shape` metric.

To add a difference metric, the following arguments need to be passed to the `add_difference_metric` method:
- `difference_metric`: The type of the metric.
- `reference`: The reference which should be used for the metric.
- `solution_path`: The path to the corresponding solution in the simulation results.

In [ ]:
comparator.add_difference_metric('RMSE', reference, 'outlet.outlet', components=["ProteinA"])
comparator.add_difference_metric('PeakPosition', reference, 'outlet.outlet', components=["ProteinA"])

## Reference Model

Now, we need our prepared model to compare to the experimental data.

In [ ]:
from CADETProcess.simulator import Cadet

simulator = Cadet()
simulation_results = simulator.simulate(process)

The difference can also be visualized:

In [ ]:
comparator.plot_comparison(simulation_results)

And we can calculate the exact value of the difference:

In [ ]:
comparator.evaluate(simulation_results)

The comparison shows that there is still a large discrepancy between simulation and experiment.

## Optimization
To find the binding_strength with the best agreement between simulation and data, we can screen some porosities and compare them to our data:

In [ ]:
import numpy as np
scan_simulation_results = {}
metrics = {}
kas = np.linspace(1e-5, 1e-2, 21)
for ka in kas:
    process.flow_sheet.column.binding_model.adsorption_rate[1] = ka
    simulation_results = simulator.simulate(process)
    scan_simulation_results[ka] = simulation_results
    metrics[ka] = comparator.evaluate(simulation_results)[0]

In [ ]:
%matplotlib ipympl
from ipywidgets import interact, interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt
from scipy.interpolate import PchipInterpolator

sim_res = scan_simulation_results[kas[0]]

reference_interpolated = PchipInterpolator(reference.time, reference.solution[:, 1])(sim_res.time_complete)

fig, (ax, ax_score) = plt.subplots(1,2, figsize=(15,10))


ax_score.bar(kas, metrics.values(), color="red", width=kas[1]-kas[0], alpha=0.6)
vline = ax_score.axvline(x=1e-5, linestyle=":", color="grey")
ax_score.set_ylabel("SSE [-]")
ax_score.set_xlabel("porosity [-]")
plt.tight_layout()

# Visualization
def graph_column(porosity=1e-5):
    ax.clear()
    sim_res = scan_simulation_results[porosity]
    ax.fill_between(sim_res.time_complete, reference_interpolated, sim_res.solution.outlet.outlet.solution_original[:, 1], color="red", alpha=0.6)
    line_sim = ax.plot(sim_res.time_complete, sim_res.solution.outlet.outlet.solution_original[:, 1])[0]
    line_ref = ax.plot(reference.time, reference.solution[:, 1], ":", color="black")
    vline.set_xdata([porosity, porosity])


style = {'description_width': 'initial'}
interact(graph_column, porosity=widgets.SelectionSlider(layout={'width': '800px'}, style=style, description='porosity', options = kas))

Instead of manually adjusting these parameters, an `OptimizationProblem` can be set up which automatically determines the parameter values.
For this purpose, an `OptimimizationProblem` is defined and the process is added as an evaluation object.

In [ ]:
from CADETProcess.optimization import OptimizationProblem

optimization_problem = OptimizationProblem("sma_binding")
optimization_problem.add_evaluation_object(process)

Then, the optimization variables are added.
Note, the parameter path associates the variable with the parameter of the corresponding column unit operation.

In [ ]:
optimization_problem.add_variable(
    "ka",
    parameter_path="flow_sheet.column.binding_model.adsorption_rate",
    lb=1e-5,
    ub=1e-2,
    transform="auto",
    indices=(1),
)
optimization_problem.add_variable(
    "kd",
    parameter_path="flow_sheet.column.binding_model.desorption_rate",
    lb=1e-8,
    ub=1e1,
    transform="auto",
    indices=(1),
)

optimization_problem.add_variable(
    "keq",
    parameter_path=None,
    evaluation_objects=None,
    lb=1e-3,
    ub=1e3,
    transform="auto",
    indices=(1),
)

def kd_from_ka_and_keq(ka, keq):
    kd = ka / keq
    return kd

optimization_problem.add_variable_dependency("kd", ["ka", "keq"], kd_from_ka_and_keq)

optimization_problem.add_variable(
    "characteristic_charge",
    parameter_path="flow_sheet.column.binding_model.characteristic_charge",
    lb=1,
    ub=10,
    transform="auto",
    indices=(1),
)

Before the difference metrics, which we want to minimize, the `Process` needs to be simulated.
For this purpose, register the `Cadet` simulator instance as an evaluator.

In [ ]:
from CADETProcess.simulator import Cadet
simulator = Cadet()

optimization_problem.add_evaluator(simulator)

Now, when adding the `Comparator` (which determines the difference metrics) as objective function, the simulator can be added to the `required` list.
Note that the number of metrics needs to be passed as `n_objectives`.

In [ ]:
optimization_problem.add_objective(
    comparator,
    n_objectives=comparator.n_metrics,
    requires=[simulator]
)

## Callbacks
A `callback` function is a user function that is called periodically by the optimizer in order to allow the user to query the state of the optimization.
For example, a simple user callback function might be used to plot results.
The function is called after each iteration for all best individuals at that state.

The callback signature may include any of the following arguments:
- `results`: obj

    x or final result of evaluation toolchain.
- `individual`: {class}`Individual`, optional

    Information about current step of optimzer.
- `evaluation_object`: obj, optional

    Current evaluation object.
- `callbacks_dir`: Path, optional

    Path to store results.

In [ ]:
from CADETProcess.optimization import U_NSGA3

optimizer = U_NSGA3()
optimizer.n_cores = 12
optimizer.pop_size = 64
optimizer.n_max_gen = 10

In [ ]:
def callback(simulation_results, individual, evaluation_object, callbacks_dir='./'):
    comparator.plot_comparison(
        simulation_results,
        file_name=f'{callbacks_dir}/{individual.id}_{evaluation_object}_comparison.png',
        show=False
    )

optimization_problem.add_callback(callback, requires=[simulator])

## Optimizer

A couple of optimizers are available in **CADET-Process**.
Depending on the problem at hand, some optimizers might outperform others.
Generally, `U_NSGA3`, a genetic algorithm, is a robust choice.
While not necessarily the most efficient, it usually manages to handle complex problems with multiple dimensions, constraints, and objectives.
Here, we limit the number of cores, the population size, as well as the maximum number of generations.

In [ ]:
from CADETProcess.optimization import U_NSGA3

In [ ]:
optimizer = U_NSGA3()
optimizer.n_cores = 4
optimizer.n_max_gen = 4
optimizer.pop_size = 32

In [ ]:
optimization_results = optimizer.optimize(optimization_problem)

### Optimization Progress and Results

The `OptimizationResults` which are returned contain information about the progress of the optimization.
For example, the attributes `x` and `f` contain the final value(s) of parameters and the objective function.

After optimization, several figures can be plotted to vizualize the results.
For example, the convergence plot shows how the function value changes with the number of evaluations.

The `plot_objectives` method shows the objective function values of all evaluated individuals.
Here, lighter color represent later evaluations.
Note that by default the values are plotted on a log scale if they span many orders of magnitude.
To disable this, set `autoscale=False`.

Note that more figures are created for constrained optimization, as well as multi-objective optimization.
All figures are also saved automatically in the `working_directory`.
Moreover, results are stored in a `.csv` file.
- The `results_all.csv` file contains information about all evaluated individuals.
- The `results_last.csv` file contains information about the last generation of evaluated individuals.
- The `results_pareto.csv` file contains only the best individual(s).

We can also look at the callbacks that were generated.